In [ ]:
# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix
from sklearn.ensemble import StackingClassifier, RandomForestClassifier
import xgboost as xgb
import lightgbm as lgb
from sklearn.decomposition import PCA
import shap
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, BatchNormalization, Dropout

# Set plot style and size
plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = (12, 8)

# For reproducibility
np.random.seed(42)
tf.random.set_seed(42)


In [ ]:
# Load the preprocessed data
print("Loading preprocessed data...")
try:
    with open('data/preprocessed_data.pkl', 'rb') as f:
        preprocessed_data = pickle.load(f)
        
    X_train = preprocessed_data['X_train']
    X_test = preprocessed_data['X_test']
    y_train = preprocessed_data['y_train']
    y_test = preprocessed_data['y_test']
    feature_names = preprocessed_data['feature_names']
    
    print(f"Data loaded successfully. Training shape: {X_train.shape}, Test shape: {X_test.shape}")
    print(f"Selected features: {feature_names}")
except Exception as e:
    print(f"Error loading preprocessed data: {e}")


In [ ]:
# Apply PCA for feature compression
print("Applying PCA for dimensionality reduction...")

# We'll try different numbers of components
for n_components in [5, 7, 9]:
    pca = PCA(n_components=n_components)
    X_train_pca = pca.fit_transform(X_train)
    X_test_pca = pca.transform(X_test)
    
    # Print explained variance ratio
    explained_variance = np.sum(pca.explained_variance_ratio_)
    print(f"PCA with {n_components} components explains {explained_variance:.2%} of variance")
    
    # Visualize the cumulative explained variance
    plt.figure(figsize=(10, 6))
    plt.plot(np.cumsum(pca.explained_variance_ratio_), marker='o')
    plt.axhline(y=0.95, color='r', linestyle='--', label='95% Explained Variance')
    plt.axvline(x=n_components, color='g', linestyle='--', label=f'{n_components} Components')
    plt.xlabel('Number of Components')
    plt.ylabel('Cumulative Explained Variance')
    plt.title(f'Explained Variance vs. Number of Components (n={n_components})')
    plt.legend()
    plt.grid(True)
    plt.show()

# Select the best PCA model (let's go with 7 components for a balance)
pca = PCA(n_components=7)
X_train_pca = pca.fit_transform(X_train)
X_test_pca = pca.transform(X_test)

print("\nComponent loadings (feature importance in each principal component):")
loadings_df = pd.DataFrame(pca.components_.T, index=X_train.columns)
loadings_df.head()


In [ ]:
# Define and train an autoencoder for feature compression
print("Training autoencoder for nonlinear dimensionality reduction...")

# Define the input dimension and encoding dimension
input_dim = X_train.shape[1]
encoding_dim = 7  # Same as PCA for comparison

# Build the autoencoder model
# Input layer
input_layer = Input(shape=(input_dim,))

# Encoder
encoder = Dense(12, activation='relu')(input_layer)
encoder = BatchNormalization()(encoder)
encoder = Dense(encoding_dim, activation='relu', name='bottleneck')(encoder)

# Decoder
decoder = Dense(12, activation='relu')(encoder)
decoder = BatchNormalization()(decoder)
decoder = Dense(input_dim, activation='linear')(decoder)

# Create the autoencoder model
autoencoder = Model(inputs=input_layer, outputs=decoder)
autoencoder.compile(optimizer='adam', loss='mse')

# Print model summary
autoencoder.summary()

# Train the autoencoder
history = autoencoder.fit(
    X_train, X_train,
    epochs=50,
    batch_size=256,
    shuffle=True,
    validation_split=0.2,
    verbose=1
)

# Plot training history
plt.figure(figsize=(10, 6))
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Autoencoder Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.grid(True)
plt.show()

# Create an encoder model to extract the bottleneck features
encoder_model = Model(inputs=autoencoder.input, outputs=autoencoder.get_layer('bottleneck').output)

# Get encoded features for train and test data
X_train_encoded = encoder_model.predict(X_train)
X_test_encoded = encoder_model.predict(X_test)

print(f"Original feature shape: {X_train.shape}")
print(f"Encoded feature shape: {X_train_encoded.shape}")


In [ ]:
# Function to evaluate models
def evaluate_model(model, X_train, X_test, y_train, y_test, model_name="Model"):
    """Evaluate model performance with multiple metrics"""
    # Train the model
    model.fit(X_train, y_train)
    
    # Make predictions
    y_pred = model.predict(X_test)
    
    # Calculate metrics
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    
    # For ROC AUC, we need probability predictions
    try:
        y_prob = model.predict_proba(X_test)[:, 1]
        auc = roc_auc_score(y_test, y_prob)
    except AttributeError:
        # If predict_proba is not available (e.g., for some models)
        auc = None
    
    # Print the results
    print(f"\n{model_name} Performance:")
    print(f"Accuracy: {accuracy:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"F1 Score: {f1:.4f}")
    if auc is not None:
        print(f"ROC AUC: {auc:.4f}")
        
    # Create confusion matrix
    cm = confusion_matrix(y_test, y_pred)
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', cbar=False)
    plt.title(f'{model_name} Confusion Matrix')
    plt.ylabel('Actual')
    plt.xlabel('Predicted')
    plt.show()
    
    # Return model and metrics
    return {
        'model': model,
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1': f1,
        'auc': auc
    }

# Initialize the base models with default parameters
print("Initializing base models...")

# XGBoost model
xgb_model = xgb.XGBClassifier(
    objective='binary:logistic',
    random_state=42,
    use_label_encoder=False,
    eval_metric='logloss'
)

# LightGBM model
lgb_model = lgb.LGBMClassifier(
    objective='binary',
    random_state=42
)

# Train and evaluate XGBoost
print("\nTraining and evaluating XGBoost model...")
xgb_results = evaluate_model(xgb_model, X_train, X_test, y_train, y_test, "XGBoost")

# Train and evaluate LightGBM
print("\nTraining and evaluating LightGBM model...")
lgb_results = evaluate_model(lgb_model, X_train, X_test, y_train, y_test, "LightGBM")


In [ ]:
# Train and evaluate models with PCA features
print("\nTraining models with PCA features...")

# XGBoost with PCA
print("\nTraining and evaluating XGBoost with PCA features...")
xgb_pca_results = evaluate_model(xgb.XGBClassifier(random_state=42, use_label_encoder=False, eval_metric='logloss'), 
                               X_train_pca, X_test_pca, y_train, y_test, "XGBoost with PCA")

# LightGBM with PCA
print("\nTraining and evaluating LightGBM with PCA features...")
lgb_pca_results = evaluate_model(lgb.LGBMClassifier(random_state=42), 
                              X_train_pca, X_test_pca, y_train, y_test, "LightGBM with PCA")


In [ ]:
# Train and evaluate models with autoencoder features
print("\nTraining models with autoencoder features...")

# XGBoost with autoencoder features
print("\nTraining and evaluating XGBoost with autoencoder features...")
xgb_ae_results = evaluate_model(xgb.XGBClassifier(random_state=42, use_label_encoder=False, eval_metric='logloss'), 
                             X_train_encoded, X_test_encoded, y_train, y_test, "XGBoost with Autoencoder")

# LightGBM with autoencoder features
print("\nTraining and evaluating LightGBM with autoencoder features...")
lgb_ae_results = evaluate_model(lgb.LGBMClassifier(random_state=42), 
                            X_train_encoded, X_test_encoded, y_train, y_test, "LightGBM with Autoencoder")


In [ ]:
# Create a stacked ensemble model
print("\nBuilding stacked ensemble model...")

# Define base models
estimators = [
    ('xgb', xgb.XGBClassifier(random_state=42, use_label_encoder=False, eval_metric='logloss')),
    ('lgb', lgb.LGBMClassifier(random_state=42))
]

# Define stacking classifier with a meta-learner
stacked_model = StackingClassifier(
    estimators=estimators,
    final_estimator=RandomForestClassifier(n_estimators=100, random_state=42),
    cv=5,  # 5-fold cross-validation for training the meta-learner
    stack_method='auto',  # Auto-detect whether to use predict_proba or decision_function
    verbose=1
)

# Train and evaluate stacked model
print("\nTraining and evaluating stacked ensemble model...")
stacked_results = evaluate_model(stacked_model, X_train, X_test, y_train, y_test, "Stacked Ensemble")

# Compare all model results
models = ['XGBoost', 'LightGBM', 'XGBoost+PCA', 'LightGBM+PCA', 
         'XGBoost+Autoencoder', 'LightGBM+Autoencoder', 'Stacked Ensemble']
results = [xgb_results, lgb_results, xgb_pca_results, lgb_pca_results,
          xgb_ae_results, lgb_ae_results, stacked_results]

# Create comparison dataframe
comparison_df = pd.DataFrame({
    'Model': models,
    'Accuracy': [r['accuracy'] for r in results],
    'Precision': [r['precision'] for r in results],
    'Recall': [r['recall'] for r in results],
    'F1 Score': [r['f1'] for r in results],
    'AUC': [r['auc'] for r in results]
})

# Display model comparison
print("\n=== Model Performance Comparison ===")
comparison_df.set_index('Model').sort_values('F1 Score', ascending=False)


In [ ]:
# Use SHAP to explain XGBoost predictions
print("\nCalculating SHAP values for XGBoost model...")

# Create explainer
explainer = shap.TreeExplainer(xgb_results['model'])

# Calculate SHAP values (sample 1000 data points for visualization if dataset is large)
if len(X_test) > 1000:
    shap_values = explainer.shap_values(X_test.sample(1000, random_state=42))
    X_test_sample = X_test.sample(1000, random_state=42)
else:
    shap_values = explainer.shap_values(X_test)
    X_test_sample = X_test

# Summary plot
plt.figure(figsize=(12, 10))
shap.summary_plot(shap_values, X_test_sample, feature_names=X_test.columns.tolist())
plt.title('SHAP Summary Plot for XGBoost Model')
plt.tight_layout()
plt.show()

# Bar plot of mean absolute SHAP values
plt.figure(figsize=(12, 10))
shap.summary_plot(shap_values, X_test_sample, feature_names=X_test.columns.tolist(), plot_type="bar")
plt.title('SHAP Feature Importance Plot')
plt.tight_layout()
plt.show()


In [ ]:
# Detailed SHAP analysis for top features
print("\nDetailed SHAP analysis for top features...")

# Calculate mean absolute SHAP values to identify top features
mean_abs_shap = np.abs(shap_values).mean(0)
feature_importance = pd.DataFrame(list(zip(X_test.columns, mean_abs_shap)),
                                columns=['Feature', 'SHAP_Importance'])
feature_importance = feature_importance.sort_values('SHAP_Importance', ascending=False)

# Display top 5 features
print("Top 5 features by SHAP importance:")
print(feature_importance.head(5))

# Create dependence plots for top 5 features
top_features = feature_importance.head(5)['Feature'].values
for feature in top_features:
    plt.figure(figsize=(12, 8))
    feature_idx = list(X_test.columns).index(feature)
    shap.dependence_plot(feature_idx, shap_values, X_test_sample, feature_names=X_test.columns)
    plt.title(f'SHAP Dependence Plot for {feature}')
    plt.tight_layout()
    plt.show()

# Create SHAP force plots for a few examples
print("\nSHAP force plots for individual examples:")
for i in range(3):  # Show 3 examples
    plt.figure(figsize=(14, 4))
    shap.force_plot(explainer.expected_value, shap_values[i,:], X_test_sample.iloc[i,:], 
                  feature_names=X_test.columns, matplotlib=True)
    plt.title(f'SHAP Force Plot for Example {i+1}')
    plt.tight_layout()
    plt.show()


In [ ]:
# Use SHAP to explain LightGBM predictions
print("\nCalculating SHAP values for LightGBM model...")

# Create explainer
explainer_lgb = shap.TreeExplainer(lgb_results['model'])

# Calculate SHAP values
if len(X_test) > 1000:
    shap_values_lgb = explainer_lgb.shap_values(X_test.sample(1000, random_state=42))
    X_test_sample = X_test.sample(1000, random_state=42)
else:
    shap_values_lgb = explainer_lgb.shap_values(X_test)
    X_test_sample = X_test

# Summary plot
plt.figure(figsize=(12, 10))
shap.summary_plot(shap_values_lgb, X_test_sample, feature_names=X_test.columns.tolist())
plt.title('SHAP Summary Plot for LightGBM Model')
plt.tight_layout()
plt.show()

# Compare feature importance between XGBoost and LightGBM
# Calculate mean absolute SHAP values for LightGBM
mean_abs_shap_lgb = np.abs(shap_values_lgb).mean(0)
feature_importance_lgb = pd.DataFrame(list(zip(X_test.columns, mean_abs_shap_lgb)),
                                   columns=['Feature', 'SHAP_Importance_LGB'])

# Merge with XGBoost importance
feature_importance_merged = feature_importance.merge(feature_importance_lgb, on='Feature')
feature_importance_merged = feature_importance_merged.sort_values('SHAP_Importance', ascending=False)

# Display comparison
print("\nFeature importance comparison between XGBoost and LightGBM:")
print(feature_importance_merged.head(10))

# Plot comparison
plt.figure(figsize=(12, 10))
feature_importance_merged_top10 = feature_importance_merged.head(10)

# Create a plot for comparison
fig, ax = plt.subplots(figsize=(12, 8))

x = np.arange(len(feature_importance_merged_top10))
width = 0.35

ax.bar(x - width/2, feature_importance_merged_top10['SHAP_Importance'], width, label='XGBoost')
ax.bar(x + width/2, feature_importance_merged_top10['SHAP_Importance_LGB'], width, label='LightGBM')

ax.set_xticks(x)
ax.set_xticklabels(feature_importance_merged_top10['Feature'], rotation=45, ha='right')
ax.legend()
ax.set_title('Feature Importance Comparison: XGBoost vs LightGBM')
ax.set_ylabel('Mean |SHAP Value|')

plt.tight_layout()
plt.show()


In [ ]:
# Choose the best performing model based on F1-Score
best_model_idx = comparison_df['F1 Score'].idxmax()
best_model_name = comparison_df.loc[best_model_idx, 'Model']
best_model_f1 = comparison_df.loc[best_model_idx, 'F1 Score']

print(f"The best performing model is {best_model_name} with an F1 Score of {best_model_f1:.4f}")

# Get all model metrics
print("\nPerformance metrics of all models:")
comparison_df = comparison_df.sort_values('F1 Score', ascending=False).reset_index(drop=True)
comparison_df

# Visualize comparison
plt.figure(figsize=(14, 8))
metrics = ['Accuracy', 'Precision', 'Recall', 'F1 Score', 'AUC']

# Melt the dataframe for easier plotting
melted_df = pd.melt(comparison_df, id_vars=['Model'], value_vars=metrics,
                    var_name='Metric', value_name='Score')

# Plot grouped bar chart
sns.barplot(data=melted_df, x='Model', y='Score', hue='Metric')
plt.title('Performance Comparison Across Models', fontsize=16)
plt.xticks(rotation=45, ha='right')
plt.ylim(0.5, 1.0)  # Set y-axis to focus on the important range
plt.tight_layout()
plt.show()

# Summary of findings
print("\n--- Key Findings ---")
print(f"1. Best model: {best_model_name} (F1 Score: {best_model_f1:.4f})")

# Top features from SHAP analysis
print("\n2. Top 5 most important features (based on SHAP values):")
for i, row in feature_importance.head(5).iterrows():
    print(f"   - {row['Feature']}: {row['SHAP_Importance']:.4f}")

# Compare dimensionality reduction approaches
print("\n3. Dimensionality Reduction Impact:")
original_f1 = max(xgb_results['f1'], lgb_results['f1'])
pca_f1 = max(xgb_pca_results['f1'], lgb_pca_results['f1'])
autoencoder_f1 = max(xgb_ae_results['f1'], lgb_ae_results['f1'])

print(f"   - Original features: F1 Score = {original_f1:.4f}")
print(f"   - PCA features: F1 Score = {pca_f1:.4f} (Diff: {pca_f1 - original_f1:.4f})")
print(f"   - Autoencoder features: F1 Score = {autoencoder_f1:.4f} (Diff: {autoencoder_f1 - original_f1:.4f})")


In [ ]:
# Save the best model based on the comparison
import joblib
import os

# Create models directory if it doesn't exist
os.makedirs('models', exist_ok=True)

# Map model names to their result dictionaries
model_results_map = {
    'XGBoost': xgb_results,
    'LightGBM': lgb_results,
    'XGBoost with PCA': xgb_pca_results,
    'LightGBM with PCA': lgb_pca_results,
    'XGBoost with Autoencoder': xgb_ae_results,
    'LightGBM with Autoencoder': lgb_ae_results,
    'Stacked Ensemble': stacked_results
}

# Get the best model
best_model_result = model_results_map.get(best_model_name)

if best_model_result:
    # Save the model
    best_model = best_model_result['model']
    model_path = os.path.join('models', f"{best_model_name.replace(' ', '_').lower()}.pkl")
    joblib.dump(best_model, model_path)
    
    # Save additional info if needed (for PCA or Autoencoder)
    if 'PCA' in best_model_name:
        joblib.dump(pca, os.path.join('models', 'pca_transformer.pkl'))
    elif 'Autoencoder' in best_model_name:
        encoder_model.save(os.path.join('models', 'autoencoder_encoder.h5'))
        
    # Also save the stacked ensemble model for deployment
    joblib.dump(stacked_results['model'], os.path.join('models', 'stacked_ensemble.pkl'))
    
    print(f"Best model ({best_model_name}) saved to {model_path}")
    print(f"Stacked ensemble model also saved to models/stacked_ensemble.pkl")
else:
    print("Could not find the best model to save.")

# Save feature importance information
feature_importance.to_csv(os.path.join('models', 'feature_importance.csv'), index=False)
print("Feature importance information saved to models/feature_importance.csv")
